In [8]:
cd '/Users/admin/Jedha/'

/Users/admin/Jedha


In [10]:
import pandas as pd 
import numpy as np
from langdetect import detect
import seaborn as sns
import io 
from s3_credentials import *

In [ ]:
#Pull subdatasets

# real
obj = s3.Object('jedha-fake-reviews-project', "datasets/real_reviews_raw.csv")
real = pd.read_csv(io.BytesIO(obj.get()['Body'].read()), low_memory = False, index_col=0)

# fake
obj = s3.Object('jedha-fake-reviews-project', "datasets/fake_reviews_raw.csv")
fake = pd.read_csv(io.BytesIO(obj.get()['Body'].read()), low_memory = False, index_col=0)


In [ ]:
columns_list_ordered = real.columns.values.tolist()
columns_list_ordered.append('url_not_recommended')

pivot_table = real.loc[:, ['url','restaurant_average_rating','restaurant_reviews_count','restaurant_expensiveness','restaurant_name']].drop_duplicates('url')

fake_merged = (pd.merge(fake, pivot_table, left_on='url', right_on = 'url', how='left'))
fake_dataset_reworked = fake_merged.loc[:, columns_list_ordered]

dataset = pd.concat([real, fake_dataset_reworked])
dataset = full_dataset.reset_index(drop = True)
dataset = full_dataset.drop_duplicates()
dataset['url_not_recommended'] = full_dataset['url'].apply(lambda url: url.replace("biz/", "not_recommended_reviews/"))

In [ ]:
# upload new dataset 
# set path and bucket name
PATH = "datasets/full_dataset.csv"
bucket = s3.Bucket(name = "jedha-fake-reviews-project")
# export dataset as csv
data = full_dataset.to_csv()
put_object = bucket.put_object(ACL='private', Key= PATH, Body=data)
#check 
for obj in bucket.objects.all():
    print(obj.key)